In [1]:
#!/usr/bin/python

#setup logging
import logging
logging.basicConfig(format='%(asctime)s:%(levelname)s:\t%(message)s', level=logging.DEBUG, datefmt='%m-%d %H:%M:%S')
logger = logging.getLogger('simple_example')
#logger.setLevel(logging.DEBUG)

pdebug = lambda x: logger.debug(x)
pinfo = lambda x: logger.info(x)
perror = lambda x: logger.error(x)
pexception = lambda x: logger.critical(x)

In [2]:
import threading
import time
from queue import Queue
from redis import Redis
import multiprocessing

exitFlag = 0

conn = Redis(host='redis', port=6379, db=0, charset="utf-8", decode_responses=True)

class myThread (threading.Thread):
    def __init__(self, manager, name, callback, pubsub=True, q="default"):
        threading.Thread.__init__(self)
        self.threadID = manager.threadID
        self.name = name
        self.callback = callback
        self.pubsub = pubsub
        self.queue = q
        self.manager = manager
        
    def run(self):
        pdebug("Starting " + self.name)
        if self.pubsub:
            self.thread_function(self.callback)
        else:
            self.thread_job(self.callback, self.queue)
        pdebug("Exiting " + self.name)
        
    def thread_function(self, callback):
        pubsub = conn.pubsub()
        pubsub.subscribe([self.name+'/cmd', self.name+'/data'])
        
        pubsub.get_message(self.name+'/cmd')
        pubsub.get_message(self.name+'/data')

        for item in pubsub.listen():
            channel = item['channel'].split('/')[1]
            data = item['data']
            pdebug(self.name+':'+channel)
            if channel== 'data':
                callback(self.manager, data)
            elif channel== 'cmd' and data == 'stop':
                pubsub.unsubscribe()
                break
                
    def thread_job(self, callback, queue):
        #pdebug(conn.rpop(queue))
        callback(queue)
        
        
        
jobs = []
class threadManager():
    def __init__(self, name, thread_list, callback_list):
        self.threads = []
        self.name = name
        self.threadList = thread_list
        self.threadCallback = callback_list
        self.threadID = 1
        
        self.job = multiprocessing.Process(target=self.init)
        jobs.append(self.job)
        self.job.start()
        
    def init(self):
        # Create new threads
        for tName in self.threadList:
            self.add(tName, self.threadCallback[self.threadID-1])

        # Wait for all threads to complete
        #for t in self.threads:
        #    t.join()
        #pinfo("Exiting Main Thread")
        
    def add(self, name, callback, pubsub=True, q="default"):
        thread = myThread(self, name, callback, pubsub, q)
        thread.start()
        self.threads.append(thread)
        self.threadID += 1

In [3]:
def hello_world1(manager, data):
    pdebug("1: "+ str(data))
    
def hello_world2(manager, data):
    pdebug("2: "+ str(data))
    

tradeManager = threadManager("tradeManager", ["order", "trade"], [hello_world1, hello_world2])


02-18 21:10:06:DEBUG:	Starting order
02-18 21:10:06:DEBUG:	Starting trade
02-18 21:10:06:DEBUG:	order:cmd
02-18 21:10:06:DEBUG:	trade:cmd
02-18 21:10:06:DEBUG:	Exiting order
02-18 21:10:06:DEBUG:	Exiting trade


In [4]:
tradeManager.add("log",hello_world1)

02-18 21:10:06:DEBUG:	Starting log


In [5]:

conn.publish('order/cmd','stop')
conn.publish('trade/cmd','stop')
conn.publish('log/cmd','stop')


02-18 21:10:06:DEBUG:	log:cmd
02-18 21:10:06:DEBUG:	Exiting log


1

In [6]:
'''
# order/data
{
'stock':'NIFTY',
'qty':'12',
'type':'BO',
'SL':'1234',
'Target':'1234',
'Price':'1234'
}


'''
import pandas as pd
import json

def order_job(queue):
    data = conn.rpop(queue)
    order_df = pd.read_json(data)
    
    stock = order_df['stock'][0]
    state=stock+'_state'
    
    pdebug('order_job: start: {}\n{}'.format(stock, order_df))
    
    pubsub = conn.pubsub()
    pubsub.subscribe([stock+'/cmd'])
    
    for item in pubsub.listen():
        data = item['data']
        pdebug('order_job: running: {}'.format(stock))
        conn.set(state, 'running')
        
        if data == "abort":
            conn.delete(state)
            pubsub.unsubscribe()
            break
        elif data == "info":
            pinfo("{} : {}\n {}\n".format(stock, conn.get(state), order_df))
    conn.delete(state)
            


#order_queue_lock = r.lock('order_queue_lock')
orderManager = ""
def order_handler(manager, data):
    pdebug('order_handler: {}'.format(data))
    stock = pd.read_json(data)['stock']
    state = stock[0]+'_state'
    if conn.get(state) == None:
        #Kex does not exist
        conn.lpush('order_queue', data)
        conn.set(state,'init')
        manager.add("order_job_"+stock[0],order_job, False, 'order_queue')
    else:
        for t in manager.threads:
            print(t.name)
        pinfo('order_handler: Trade in progress: publish command to {}/cmd'.format(pd.read_json(data)['stock'][0]))
        

In [7]:
conn.delete('order_queue')
orderManager = threadManager("orderManager", ["order_handler"], [order_handler])    

02-18 21:10:07:DEBUG:	Starting order_handler
02-18 21:10:12:DEBUG:	order_handler:data
02-18 21:10:13:DEBUG:	order_handler: [{"cmd":"Buy","stock":"INFY","qty":"12","type":"BO","SL":"1234","Target":"1234","Price":"1234","state":"idle"}]
02-18 21:10:13:DEBUG:	Starting order_job_INFY
02-18 21:10:13:DEBUG:	order_job: start: INFY
   cmd stock  qty type    SL  Target  Price state
0  Buy  INFY   12   BO  1234    1234   1234  idle
02-18 21:10:13:DEBUG:	order_job: running: INFY
02-18 21:10:29:DEBUG:	order_job: running: INFY
02-18 21:10:29:INFO:	INFY : running
    cmd stock  qty type    SL  Target  Price state
0  Buy  INFY   12   BO  1234    1234   1234  idle

02-18 21:10:59:DEBUG:	order_job: running: INFY
02-18 21:10:59:DEBUG:	Exiting order_job_INFY
02-18 21:11:05:DEBUG:	order_handler:data
02-18 21:11:05:DEBUG:	order_handler: [{"cmd":"Buy","stock":"INFY","qty":"12","type":"BO","SL":"1234","Target":"1234","Price":"1234","state":"idle"}]
02-18 21:11:05:DEBUG:	Starting order_job_INFY
02-18 21:11:05

order_handler
order_job_INFY
order_job_INFY


02-18 21:11:20:INFO:	order_handler: Trade in progress: publish command to INFY/cmd
02-18 21:11:21:DEBUG:	order_job: running: INFY
02-18 21:11:21:INFO:	INFY : running
    cmd stock  qty type    SL  Target  Price state
0  Buy  INFY   12   BO  1234    1234   1234  idle

02-18 21:11:58:DEBUG:	order_handler:data
02-18 21:11:58:DEBUG:	order_handler: [{"cmd":"Buy","stock":"INFY","qty":"12","type":"BO","SL":"1234","Target":"1234","Price":"1234","state":"idle"}]


order_handler
order_job_INFY
order_job_INFY


02-18 21:11:58:INFO:	order_handler: Trade in progress: publish command to INFY/cmd
02-18 21:12:26:DEBUG:	order_job: running: INFY
02-18 21:12:26:INFO:	INFY : running
    cmd stock  qty type    SL  Target  Price state
0  Buy  INFY   12   BO  1234    1234   1234  idle

02-18 21:12:32:DEBUG:	order_job: running: INFY
02-18 21:12:32:DEBUG:	Exiting order_job_INFY
02-18 21:12:39:DEBUG:	order_handler:data
02-18 21:12:39:DEBUG:	order_handler: [{"cmd":"Buy","stock":"INFY","qty":"12","type":"BO","SL":"1234","Target":"1234","Price":"1234","state":"idle"}]
02-18 21:12:39:DEBUG:	Starting order_job_INFY
02-18 21:12:39:DEBUG:	order_job: start: INFY
   cmd stock  qty type    SL  Target  Price state
0  Buy  INFY   12   BO  1234    1234   1234  idle
02-18 21:12:39:DEBUG:	order_job: running: INFY
02-18 21:12:44:DEBUG:	order_handler:data
02-18 21:12:44:DEBUG:	order_handler: [{"cmd":"Buy","stock":"INFY","qty":"12","type":"BO","SL":"1234","Target":"1234","Price":"1234","state":"idle"}]


order_handler
order_job_INFY
order_job_INFY
order_job_INFY


02-18 21:12:44:INFO:	order_handler: Trade in progress: publish command to INFY/cmd
02-18 21:13:19:DEBUG:	order_job: running: INFY
02-18 21:13:19:DEBUG:	Exiting order_job_INFY
02-18 21:13:25:DEBUG:	order_handler:cmd
02-18 21:13:25:DEBUG:	Exiting order_handler


In [8]:
import pandas as pd
import json

data={
    'cmd':'Buy',
    'stock':'INFY',
    'qty':'12',
    'type':'BO',
    'SL':'1234',
    'Target':'1234',
    'Price':'1234',
    'state':'idle'
}

df = pd.DataFrame(data=data, index=['stock'])
json_data = df.to_json(orient='records')

In [9]:
json_data

'[{"cmd":"Buy","stock":"INFY","qty":"12","type":"BO","SL":"1234","Target":"1234","Price":"1234","state":"idle"}]'

In [10]:
conn.delete("INFY_state")

1

In [23]:
conn.publish('order_handler/data',json_data)
#conn.publish('order_handler/data','swagatam')

1

In [20]:
conn.publish('INFY/cmd','info')

1

In [21]:

conn.publish('INFY/cmd','abort')

1

In [25]:
conn.publish('INFY/cmd','abort')
conn.publish('SBI/cmd','abort')
conn.publish('WIPRO/cmd','abort')

0

In [27]:
conn.publish('order_handler/cmd','stop')

1

In [28]:
conn.pubsub_channels()

[]

In [21]:
conn.pubsub_numsub('nifty/cmd')

[('nifty/cmd', 0)]

In [32]:
jobs[1].is_alive()

False